<a href="https://www.kaggle.com/code/jasmindc/great-barrier-reef-object-detection?scriptVersionId=145349838" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Great Barrier Reef Object Detection

## Initial Settings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import ast
import os
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.transforms import Bbox

pd.set_option("display.max_colwidth",None)

In [ ]:
train = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
test = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv')

## Exploring Dataset

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("train dataset Info:")
print(train.info())
print("\n")
print("test dataset Info:")
print(test.info())

In [ ]:
# Boxplot (to find Outliers)
plt.figure(figsize=(15,15))
pos = 1
for i in train.columns:
        if(type(train[i][0]) != str):
                plt.subplot(4, 3, pos)
                sns.boxplot(train[i])
                pos += 1

In [ ]:
# Correlation Matrix ( help to choose columns to drop: the most correlated!)
correlation_matrix = train.corr()
sns.heatmap(correlation_matrix, cmap="YlGnBu", annot=True)

#### No noticeable positive or negative correlation detected

In [ ]:
#check if there are null val
plt.figure(figsize=(10,10))
sns.heatmap(train.isna())

#### As we can see from the plot here are no nulls detected

In [ ]:
train.dtypes

In [ ]:
#Number of video_frame per video
train.groupby("video_id")["video_frame"].count()


In [ ]:
#plot video_id distribution ( number of images per video)
plt.figure(figsize=(10,10))
sns.histplot(data=train, x="video_id")

In [ ]:
#Adding image path from train_images folder to the dataset images (train folder)
train["image_path"] = '/kaggle/input/tensorflow-great-barrier-reef/train_images/video_'+train["video_id"].astype(str)+'/'+train["image_id"].apply(lambda x: x.split("-")[1])+".jpg"

In [ ]:
train.head()

In [ ]:
#Plotting random images to check previous step

rows, cols = 2, 2
fig, axs = plt.subplots(rows, cols, figsize=(15,10))
for i,ax in zip(train, axs.ravel()):
    random_image = random.randint(0,len(train)-1)
    img = mpimg.imread(train["image_path"][random_image])
    ax.imshow(img)
    ax.set_title(f'Image ID: {train["image_id"][random_image]}',{"fontsize": 20})

## Exploring Annotations

-  Annotations have the following format: [{'x': 645, 'y': 182, 'width': 41, 'height': 45}]
-  There can be multiple records in one list



In [ ]:
len(train["annotations"])  

In [ ]:
train["annotations"].dtype

In [ ]:
train.annotations.describe()

In [ ]:
train.annotations.unique()

In [ ]:
#calculate records with and without annotations
records_with_annotations = train[train["annotations"] != "[]"]
num_records_with_annotations = records_with_annotations["annotations"].count()

records_without_annotations = train[train["annotations"] == "[]"]
num_records_without_annotations = records_without_annotations["annotations"].count()

In [ ]:
#plot records annotation distribution
sns.barplot(x = ["record with annotations", "record without annotations"], y = [num_records_with_annotations, num_records_without_annotations], palette = "colorblind")
plt.title("Records Annotation Distribution", fontsize = 30)
plt.xlabel("Annotation", fontsize = 15)
plt.ylabel("Count", fontsize = 15)
plt.show()

In [ ]:
#Records with annotations
records_with_annotations

In [ ]:
#Number of records without annotations
print("There are "+ str(num_records_without_annotations)+ " records without annotations and "+ str(num_records_with_annotations)+ " records with annotations.")
            

In [ ]:
#Calculate the number of total annotations within the frame and adding to the data
train["no_annotations"] = train["annotations"].apply(lambda x: len(eval(x)))

In [ ]:
#Total number of annotations
tot=0
for i in train["video_frame"]:
    if train["no_annotations"][i] != 0:
        tot = tot+train["no_annotations"][i]

print("There are " + str(tot)+ " annotations in total")

#### There are a lot of records without annotations, this can make it difficult creating a good model. But, previous information are useful to check upon the model training part.

In [ ]:
#Plot annotation length distribution for records that do have annotations

sns.countplot(x=records_with_annotations["annotations"].apply(lambda x: len(x)).value_counts(),palette = "colorblind")
plt.title("Annotation Length Distribution", fontsize = 30)
plt.xlabel("Annotation", fontsize = 15)
plt.ylabel("Count", fontsize = 15)
plt.show()

In [ ]:
# Annotation distribution per video_id

video_1 = (records_with_annotations[records_with_annotations["video_id"] == 0]["annotations"]).count()
video_2 = (records_with_annotations[records_with_annotations["video_id"] == 1]["annotations"]).count()
video_3 = (records_with_annotations[records_with_annotations["video_id"] == 2]["annotations"]).count()

ax = sns.barplot(x=['Video id: 0', 'Video id: 1', 'Video id: 2'], y=[video_1, video_2, video_3])
ax.set_ylabel('Count')

#### Video with ID 2 has the least amount of annotations record.

## Detecting object with the help of a bounding box from a random image file

Each record in the annotation represents a bounding box. We can see from above exploration that one annotation list can have multiple records. So, this correspond to multiple bounding boxes.
In order to draw bounding box we must transform the annotations to list. annotations is in a string format. For bounding box the indices must be integers.

In [ ]:
#Show image and annotations if applicable
def show_image(path, annot, axs=None):
    '''Shows an image and marks any COTS annotated within the frame.
    path: full path to the .jpg image
    annot: string of the annotation for the coordinates of COTS'''
    
    # This is in case we plot only 1 image
    if axs==None:
        fig, axs = plt.subplots(figsize=(23, 8))
    
    img = plt.imread(path)
    axs.imshow(img)

    if annot:
        for a in eval(annot):
            rect = patches.Rectangle((a["x"], a["y"]), a["width"], a["height"], 
                                     linewidth=3, edgecolor="#FF6103", facecolor='none')
            axs.add_patch(rect)

    axs.axis("off")
 

In [ ]:
# Show only 1 image as example
path = list(train[train["no_annotations"]==0]["image_path"])[0]
annot = list(train[train["no_annotations"]==0]["annotations"])[0]


print("Path:",path)
print("Annotation:", annot)
print("Frame:")
show_image(path, annot, axs=None)

In [ ]:
# Show only 1 image as example
path = list(train[train["no_annotations"]==18]["image_path"])[0]
annot = list(train[train["no_annotations"]==18]["annotations"])[0]


print("Path:",path)
print("Annotation:", annot)
print("Frame:")
show_image(path, annot, axs=None)

In [ ]:
def show_multiple_images(seq_id, frame_no):
    '''Shows multiple images within a sequence.
    seq_id: a number corresponding with the sequence unique ID
    frame_no: a list containing the first and last frame to plot'''
    
    # Select image paths & their annotations
    paths = list(train[(train["sequence"]==seq_id) & 
                 (train["sequence_frame"]>=frame_no[0]) & 
                 (train["sequence_frame"]<=frame_no[1])]["image_path"])
    annotations = list(train[(train["sequence"]==seq_id) & 
                 (train["sequence_frame"]>=frame_no[0]) & 
                 (train["sequence_frame"]<=frame_no[1])]["annotations"])

    # Plot
    fig, axs = plt.subplots(2, 3, figsize=(23, 10))
    axs = axs.flatten()
    fig.suptitle(f"Showing consecutive frames for Sequence ID: {seq_id}", fontsize = 20)

    for k, (path, annot) in enumerate(zip(paths, annotations)):
        axs[k].set_title(f"Frame No: {frame_no[0]+k}", fontsize = 12)
        show_image(path, annot, axs[k])

    plt.tight_layout()
    plt.show()

In [ ]:
#example
seq_id = 44160
frame_no = [51, 56]

show_multiple_images(seq_id, frame_no)

In [ ]:
#example
seq_id = 59337
frame_no = [38, 43]

show_multiple_images(seq_id, frame_no)

In [ ]:
#compare images with the same number of annotations
def plot_comparison(no_annot, state=24):
    
    # Select image paths & their annotations
    paths_compare = list(train[train["no_annotations"]==no_annot]\
                         .sample(n=9, random_state=state)["image_path"])
    annotations_compare = list(train[train["no_annotations"]==no_annot]\
                               .sample(n=9, random_state=state)["annotations"])

    # Plot
    fig, axs = plt.subplots(3, 3, figsize=(23, 13))
    axs = axs.flatten()
    fig.suptitle(f"{no_annot} annotations", fontsize = 20)

    for k, (path, annot) in enumerate(zip(paths_compare, annotations_compare)):
        video_id = path.split("/")[4]
        frame_id = path.split("/")[-1].split(".")[0]
        
        axs[k].set_title(f"{video_id} | Frame {frame_id}",
                         fontsize = 12)
        show_image(path, annot, axs[k])

    plt.tight_layout()
    plt.show()

In [ ]:
# No annotations example
no_annot = 0
plot_comparison(no_annot, state=24)

In [ ]:
#10 annotations example
no_annot = 10
plot_comparison(no_annot, state=24)

In [ ]:
#formatting annotations
def format_annotations(x):
    '''Changes annotations from format {x, y, width, height} to {x1, y1, x2, y2}.
    x: a string of the initial format.'''
    
    annotations = eval(x)
    new_annotations = []

    if annotations:
        for annot in annotations:
            new_annotations.append([annot["x"],
                                    annot["y"],
                                    annot["x"]+annot["width"],
                                    annot["y"]+annot["height"]
                                   ])
    
    if new_annotations: return str(new_annotations)
    else: return "[]"

In [ ]:
# Create a new column with the new formated annotations
train["f_annotations"] = train["annotations"].apply(lambda x: format_annotations(x))

### We will be using Yolov8 model, so we need the right format for annotations (labels):

In [ ]:
def format_yolo_annotations(x):
    '''Changes annotations from format {x, y, width, height} to {x1, y1, x2, y2}.
    x: a string of the initial format.'''
    
    annotations = eval(x)
    new_annotations = ''
    img_h=720
    img_w=1280

    if annotations:
        for annot in annotations:
            current_category = 0
            x = annot["x"]
            y = annot["y"]
            w = annot["width"]
            h = annot["height"]
            
            # Finding midpoints
            x_centre = (x + (x+w))/2
            y_centre = (y + (y+h))/2

            # Normalization
            x_centre = x_centre / img_w
            y_centre = y_centre / img_h
            w = w / img_w
            h = h / img_h

            # Limiting upto fix number of decimal places
            x_centre = format(x_centre, '.6f')
            y_centre = format(y_centre, '.6f')
            w = format(w, '.6f')
            h = format(h, '.6f')
        
            new_annotations = new_annotations + (f"{current_category} {x_centre} {y_centre} {w} {h}\n")

    
    if new_annotations: return new_annotations
    else: return ''

In [ ]:
# Create a new column with the new formated annotations
train["yolo_annot"] = train["annotations"].apply(lambda x: format_yolo_annotations(x))

## Using Yolov8 model

In [ ]:
train.tail()

In [ ]:
#create images directory for yolov8
newpath = '/kaggle/working/images/train/'
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
#select img with annotations for .jpg
img_annot_jpg = []
for i,data in train.iterrows() :
    if not data["annotations"] == "[]":
        img_annot_jpg.append(data["image_path"])

print(len(img_annot_jpg)) #must be the same as before

In [ ]:
#copy images to right directory for yolov8
import shutil
dir = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
dest = '/kaggle/working/images/train/'

count=0

for file in os.listdir(dir):
        folder = dir + file
        for img in os.listdir(folder):
            img_path = os.path.join(folder,img)
            if img_path in img_annot_jpg:
                count=count+1
                shutil.copy(img_path, dest)
print(count) #must be the same as before

In [ ]:
#create labels directory for yolov8
newpath = '/kaggle/working/labels/train/'
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:

#create labels directory for yolov8
dest_labels = '/kaggle/working/labels/train/'
if not os.path.exists(dest_labels):
    os.makedirs(dest_labels)

count=0
#select records with annotations for .txt and copy them in the right directory
for i,data in train.iterrows() :
    if not data["annotations"] == "[]":
        count=count+1
        file =  str(data["video_frame"]) + ".txt"
        f = open(os.path.join(dest_labels,file),"w")
        f.write(data["yolo_annot"])
        f.close()
        
print(count) #must be the same as before

In [ ]:
#check specific image for annotations/labels
num= 100 #put the choosen number to be checked
path_l = '/kaggle/working/labels/train/'
path_i = '/kaggle/working/images/train/'
annot =  list(train[train["video_frame"]==num]["annotations"])[0]


label = str(num)+".txt"
image = str(num)+".jpg"
print ("Image real path: " + train["image_path"][num])
print ("Image yolo path: " + os.path.join(path_i,image))
print("Annotations: " + train["annotations"][num])
print("\n")
print("Label yolo path: " + os.path.join(path_l,label))
f = open(os.path.join(path_l, label), 'r')
print("Annotations yolo format: " +f.read())
f.close()
show_image(os.path.join(path_i,image), annot, axs=None)

In [ ]:
# Show only 1 image as example
path = '/kaggle/working/images/train/9470.jpg'
annot =  list(train[train["video_frame"]==9470]["annotations"])[0]


print("Path:",path)
print("Annotation:", annot)
print("Frame:")
show_image(path, annot, axs=None)

### Training:

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO


#Load model
model = YOLO("yolov8n.yaml") #build a new model from scratch

#Use the model
results = model.train(data = "/kaggle/input/data-settings/config.yml", epochs=100) #train the model

## Show Results

In [ ]:
for file in os.listdir('/kaggle/working/runs/detect/train/'):
    print(file)

In [ ]:
res = pd.read_csv('/kaggle/working/runs/detect/train/results.csv')
res

In [ ]:

path = '/kaggle/working/runs/detect/train/'
def show_res(file):
    path = '/kaggle/working/runs/detect/train/'
    fig, axs = plt.subplots(figsize=(23, 8))
    file_path = os.path.join(path,file)
    img = plt.imread(file_path)
    fig.tight_layout()

    axs.set_title(file, fontsize=20)
    axs.imshow(img)


for file in os.listdir(path):
    if file.endswith(('.png', '.jpg', '.jpeg')):
        show_res(file)